# A (slow) implementation of softmax regression

To first understand how everything works, we do not vectorize the operations yet (this will be the topic of TP 3).

## Loading the data

In [11]:
import numpy as np
from load_fmnist import TEXT_LABELS, load_images, load_labels
import time

IN_DIM, OUT_DIM = 784, 10

# for reproducibility
np.random.seed(42)

def show(n, width=300):
    import plotly.express as px
    fig = px.imshow(train_imgs[n], binary_string=True, width=width, title=TEXT_LABELS[train_labels[n]])
    fig.show()


In [12]:
train_imgs = load_images()
train_labels = load_labels()
val_imgs = load_images(test=True)
val_labels = load_labels(test=True)

In [13]:
show(5, width=1000)

## Make sure data has the right shape

In [14]:
train_imgs.shape, train_labels.shape

((60000, 28, 28), (60000,))

In [15]:
train_imgs_flat = train_imgs.reshape(len(train_imgs), -1)
val_imgs_flat = val_imgs.reshape(len(val_imgs), -1)
assert train_imgs_flat.shape == (60000, 784) and val_imgs_flat.shape == (10000, 784)

## Define softmax, cross entropy loss, initialization

In [16]:
def softmax(o):
    o_exp = np.exp(o - np.max(o)) # for numeric stability
    return o_exp / np.sum(o_exp)

def cross_entropy_loss(p, c):
    return -np.log(p[c] + 1e-15)

def initialize(sigma=0.01):
    return np.random.normal(0, sigma, (IN_DIM, OUT_DIM)), np.zeros(OUT_DIM)


## Compute the gradient

In [17]:
def grad(x, probs, label):
    probs_copy = probs.copy()
    probs_copy[label] -= 1
    return np.outer(x.T, probs_copy), probs_copy

## The main training function

In [18]:
def train_sgd(lr=0.01, num_epochs=10, sigma=0.01, batch_size=64, W=None, b=None, verbose=False):
    print(f"Training softmax regression with SGD, {lr=}, {batch_size=}, {num_epochs=}, non-vectorized version")

    if W is None: W, b = initialize(sigma=sigma)

    for epoch in range(num_epochs):
        t0 = time.time()
        loss = 0
        num_batches = len(train_imgs_flat) // batch_size
        for batch_imgs, batch_labels in create_batches(train_imgs_flat, train_labels, batch_size):
            grad_W, grad_b = np.zeros_like(W), np.zeros_like(b)
            for x, label in zip(batch_imgs, batch_labels):
                probs = softmax(x @ W + b)
                loss += cross_entropy_loss(probs, label) / batch_size
                grad_W_sample, grad_b_sample = grad(x, probs, label)
                grad_W += grad_W_sample / batch_size
                grad_b += grad_b_sample / batch_size
            W -= lr * grad_W
            b -= lr * grad_b

        if verbose:
            print(f"Epoch {epoch+1}, avg loss per batch: {loss / num_batches}, time: {(time.time() - t0):3f}")
    return W, b

## Creating random batches

In [19]:
def create_batches(data, labels, batch_size):
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    N = len(indices)
    indices = indices[:(N // batch_size) * batch_size]
    for s in range(0, len(indices), batch_size):
        batch_indices = indices[s:s + batch_size]
        yield data[batch_indices], labels[batch_indices]


## Training

In [20]:
W, b = train_sgd(verbose=True)

Training softmax regression with SGD, lr=0.01, batch_size=64, num_epochs=10, non-vectorized version


Epoch 1, avg loss per batch: 0.9625186890928045, time: 7.073252
Epoch 2, avg loss per batch: 0.66849999329641, time: 5.604886
Epoch 3, avg loss per batch: 0.6037729718593524, time: 5.377068
Epoch 4, avg loss per batch: 0.568718543786656, time: 5.134371
Epoch 5, avg loss per batch: 0.5459461328176514, time: 5.087615
Epoch 6, avg loss per batch: 0.5291009203129482, time: 11.246519
Epoch 7, avg loss per batch: 0.5167158406402211, time: 6.371044
Epoch 8, avg loss per batch: 0.5063440265522178, time: 8.877657
Epoch 9, avg loss per batch: 0.4979206088244604, time: 6.395602
Epoch 10, avg loss per batch: 0.49073426515662727, time: 4.985888


## Evaluating the model

In [21]:
def evaluate(W, b, ims, ys):
    correct = 0
    for x, y in zip(ims,ys):
        pred = np.argmax(softmax(x @ W + b))
        if pred == y:
            correct += 1
    return correct / len(ys)

In [22]:
val_accuracy = evaluate(W, b, val_imgs_flat, val_labels)
print(f"Validation Accuracy: {val_accuracy:.2%}")
assert val_accuracy > .8

Validation Accuracy: 82.42%
